In [1]:
import numpy as np
import cv2 as cv

from sklearn.externals import joblib
from matplotlib import pyplot as plt

import math
import pytesseract
from PIL import Image

In [2]:
clf = joblib.load('plate.pkl')

In [16]:
def drawPlate(image) :
    grayImage = cv.cvtColor(image,cv.COLOR_RGB2GRAY)
    binaryImage = cv.adaptiveThreshold(grayImage,255,cv.ADAPTIVE_THRESH_MEAN_C,cv.THRESH_BINARY,31,5)
    x,y,w,h = 0,0,0,0,
    _,contours,_ = cv.findContours(binaryImage, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE);
    for contour in contours:
        area = cv.contourArea(contour);
        if area > 1000 : 
            [x,y,w,h] = cv.boundingRect(contour);
            outputRect = image[y:y+h,x:x+w,:]
            high,width,channel = outputRect.shape
            f1 = width/high
            f2 = math.log(width*high)
            #grayScale -> Binary Threshold -> Histogram
            imageGray = cv.cvtColor(outputRect,cv.COLOR_RGB2GRAY)
            adaptive = cv.adaptiveThreshold(imageGray,255,
                                            cv.ADAPTIVE_THRESH_GAUSSIAN_C,
                                            cv.THRESH_BINARY,
                                            31,5)

            #histogram
            hist = np.histogram(adaptive)
            f3,f4 = hist[0][0],hist[0][-1]
            f3 = math.log(f3) if f3 != 0 else 0
            f4 = math.log(f4) if f4 != 0 else 0
            f5 = f3/f4 if f4 != 0 else 0   # f5 = f4 != 0 ? f3/f4 : 0 
            feature = np.asarray([f1,f2,f3,f4,f5])
            result = clf.predict([feature])[0]
            if(result == 1) :
                cv.rectangle(image, (x,y), (x+w, y+h), (0,255,0), 10);
                
    return image

In [24]:
cap = cv.VideoCapture(1)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    # Display the resulting frame
    
    frame = drawPlate(frame)
    
    cv.imshow('frame',frame)
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv.destroyAllWindows()

In [23]:
cap.release()
cv.destroyAllWindows()